In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
import librosa
from IPython.display import display
import matplotlib.pyplot as plt
from pydub import AudioSegment
import json
from sklearn import preprocessing
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [2]:
film_csv = "/Volumes/Seagate/natasha-diploma/content_data.csv"
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"

films = pd.read_csv(film_csv, index_col=None, header=0)
display(films.head())

trailers_meta = pd.read_csv(trailers_csv, index_col=None, header=0)
display(trailers_meta.head())

# df1 = pd.read_csv(film_csv, index_col=None, header=0)
# df2 = pd.read_csv(trailers_csv, index_col=None, header=0)
# trailers_meta_info = pd.merge(df1, df2, left_on='id', right_on='content_id', how='left').drop('id', axis=1)

# display(trailers_meta_info.head())

# trailers_names = []
# all_texts = []
# emotions = []
# for i in range(0, len(trailers_meta_info)):
#     url = trailers_meta_info.iloc[i]['url']
#     trailers_names.append(url[url.rfind('/')+1:-4])
#     tokenizer = RegexTokenizer()
#     text = trailers_meta_info.iloc[i]['synopsis']
#     all_texts.append(text)
    
# tokenizer = RegexTokenizer()
# tokens = tokenizer.split(''.join(all_texts))
# model = FastTextSocialNetworkModel(tokenizer=tokenizer)

# for i in range(0, len(trailers_meta_info)):
#     text = all_texts[i]
#     emotion = model.predict([text])[0]
#     emotions.append(emotion)
    
# trailers_meta_info['trailers_name'] = trailers_names
# trailers_meta_info['emotions'] = emotions
# display(trailers_meta_info.head())

# trailers_meta_info.to_csv(trailers_csv)

,id,title,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties
0,60432,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д..."
1,64219,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес..."
2,65407,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл..."
3,75149,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""..."
4,75629,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...","{ ""Аудитория"" : [""для детей 13-16 лет"",""9 лет""..."


,Unnamed: 0,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties,content_id,title_y,url,trailers_name,emotions
0,0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д...",60432,Трейлер (русский язык),http://file30.vcp.digitalaccess.ru/contents/a/...,403a42b604f504069c41d39c5a4fc2,"{'neutral': 0.5312193632125854, 'negative': 0...."
1,1,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д...",60432,Трейлер (английский язык),http://file51.vcp.digitalaccess.ru/contents/9/...,e45a4697e1e6be16f301f13197ffa4,"{'neutral': 0.5312193632125854, 'negative': 0...."
2,2,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес...",64219,Трейлер (русский язык),http://file66.vcp.digitalaccess.ru/contents/1/...,11fe2dd4cccaf0567c878e19e1811a,"{'negative': 0.4688006341457367, 'skip': 0.392..."
3,3,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл...",65407,Трейлер (русский язык),http://file55.vcp.digitalaccess.ru/contents/b/...,a5b2dd3ed41aabb5733c7cfffff0bc,"{'positive': 0.3140605390071869, 'negative': 0..."
4,4,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""...",75149,Трейлер (дублированный),http://file11.vcp.digitalaccess.ru/contents/9/...,80870ec50515107ba7034ab758cebe,"{'neutral': 0.4765896201133728, 'negative': 0...."


In [3]:
def feature_extraction(y, sr):
    hop_length = 256
    frame_length = 512
    
    energy = np.array([
        sum(abs(y[i:i+frame_length]**2))
        for i in range(0, len(y), hop_length)
    ])
    rmse = librosa.feature.rms(y, frame_length=frame_length, hop_length=hop_length, center=True)
    
    frames = range(len(energy))
    t = librosa.frames_to_time(frames, sr=sr, hop_length=hop_length)
    zero_crossings = librosa.zero_crossings(y, sr)
    zero_crossings_count = 0
    for i in range(len(zero_crossings)):
        if zero_crossings[i] == True:
            zero_crossings_count += 1
    tempo, beat_times = librosa.beat.beat_track(y, sr=sr, units='time')
    y_harmonic, y_percussive = librosa.effects.hpss(y, margin=(1.0, 5.0))
    Y = librosa.stft(y)
    Ydb = librosa.amplitude_to_db(abs(Y))
    spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]
    mfcc = librosa.feature.mfcc(y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    
    return {
        'energy': np.mean(energy),
        'tempo': tempo,
        'zero_cross_count': zero_crossings_count,
        'beat_times': np.mean(beat_times),
        'harmonic': np.mean(y_harmonic),
        'percussive': np.mean(y_percussive),
        'amplitude': np.mean(Ydb),
        'spectral_rolloff': np.mean(spectral_rolloff),
        'mfcc': np.mean(mfcc),
        'chroma_stft': np.mean(chroma_stft),
        'rmse': np.mean(rmse),
        'spec_bw': np.mean(spec_bw)
    }


def mean(array_str):
    array = array_str[1:-1].split(',')
    sum = 0
    length = 0
    for i in range(len(array)):
        if float(array[i]) != float("inf"):
            sum += float(array[i])
            length += 1
            
    return sum/length
        
        
        
def process_data(df):
    temperature = []
    color_hist = []
    brightness = []
    colorfulness = []
    for i in range(0, len(df)):
        temperature.append(mean(df.iloc[i]['temperature']))
        brightness.append(mean(df.iloc[i]['brightness']))
        colorfulness.append(mean(df.iloc[i]['colorfulness']))
        
    df['temperature'] = temperature
    df['brightness'] = brightness
    df['colorfulness'] = colorfulness
    
    return df


def rgb_to_hsv(r, g, b):
    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = (df/mx)*100
    v = mx*100
    return h, s, v


def class_hue(h):
    if 30 < h < 50:
        return 'orange'
    if 50 < h < 70:
        return 'yellow'
    if 70 < h < 160:
        return 'green'
    if 160 < h < 200:
        return 'aqua'
    if 200 < h < 240:
        return 'blue'
    if 240 < h < 300:
        return 'purple'
    if 300 < h < 330:
        return 'pink'
    else:
        return 'red'
    
    
def process_every_five(array):
    new_arr = []
    sum = 0;
    for i in range(len(array)):
        if (i % 5 != 0):
            sum += array[i]
        else:
            new_arr.append(sum)
            sum = 0
            
    return new_arr


def normalize(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    display(normalized_df.head())
    return df
    

In [4]:
img_info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
audio_info_folder = "/Volumes/Seagate/natasha-diploma/audioinfo"
audio_folder = "/Volumes/Seagate/natasha-diploma/audio_wav"
print(len(trailers_meta['trailers_name']))
for i in range(0, len(trailers_meta)):       
    name = trailers_meta.iloc[i]['trailers_name']
    img_info = img_info_folder + '/' + name + '.csv'
    audio_info = audio_info_folder + '/' + name + '.csv'
    audio = audio_folder + '/' + name + '.wav'
    
    if os.path.exists(audio) and os.path.exists(img_info):
        print(name, i)
        x, sr = librosa.load(audio)
        audio_duration = librosa.get_duration(x) * 1000

        img_df = pd.read_csv(img_info, index_col=None, header=0)
        img_df.drop(img_df.columns[img_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
        img_df.drop(img_df.columns[img_df.columns.str.contains('color_hist',case = False)],axis = 1, inplace = True)

        scenes = img_df['scene'].iloc[-1]
        dur_per_scene = audio_duration / (scenes + 1)
        
        if 'colors' in img_df:
            orange = []
            red = []
            yellow = []
            green = []
            aqua = []
            blue = []
            purple = []
            pink = []
            for i in range(0, len(img_df)):
                colors_str = img_df.iloc[i]['colors']
                colors = colors_str[1:-1].split('\n')
                for ind in range(len(colors)):
                    rgb = colors[ind].strip("[ ]").split('  ')
                    rgb = [x for x in rgb if x != '']
                    hsv = rgb_to_hsv(float(rgb[0].strip('[ ]')), float(rgb[1].strip('[ ]')), float(rgb[2].strip('[ ]')))
                    col = class_hue(hsv[0])
                    orange.append(int(col == 'orange'))
                    red.append(int(col == 'red'))
                    yellow.append(int(col == 'yellow'))
                    green.append(int(col == 'green'))
                    aqua.append(int(col == 'aqua'))
                    blue.append(int(col == 'blue'))
                    purple.append(int(col == 'purple'))
                    pink.append(int(col == 'pink'))

            img_df['orange'] = process_every_five(orange)
            img_df['red'] = process_every_five(red)
            img_df['yellow'] = process_every_five(yellow)
            img_df['green'] = process_every_five(green)
            img_df['aqua'] = process_every_five(aqua)
            img_df['blue'] = process_every_five(blue)
            img_df['purple'] = process_every_five(purple)
            img_df['pink'] = process_every_five(pink)
        
        img_df.drop(img_df.columns[img_df.columns.str.contains('colors',case = False)],axis = 1, inplace = True)
        img_df.drop(img_df.columns[img_df.columns.str.contains('centro',case = False)],axis = 1, inplace = True)
        img_df.drop(img_df.columns[img_df.columns.str.contains('zero_crossings',case = False)],axis = 1, inplace = True)
         
        img_df.to_csv(img_info)
    
        if 'tempo' in img_df:
            print("pass file: ", img_info)
            continue

        else:
            energy = []
            tempo = []
            centro = []
            zero_crossings = []
            zero_cross_count = []
            beat_times = []
            harmonic = []
            percussive = []
            amplitude = []
            spectral_rolloff = []
            mfcc = []
            chroma_stft = []
            rmse = []
            spec_bw = []
            newAudio = AudioSegment.from_wav(audio)
            for i in range(0, len(img_df)):   
                scene = int(img_df.iloc[i]['scene'])
                if (dur_per_scene * scene >= audio_duration):
                    break
                newAudio_temp = newAudio[dur_per_scene * scene:dur_per_scene * (scene + 1)]
                newAudio_temp.export('newSong.wav', format="wav")

                x_temp, sr_temp = librosa.load('newSong.wav')

                features = feature_extraction(x_temp, sr_temp)

                energy.append(features['energy'])
                tempo.append(features['tempo'])
                zero_cross_count.append(features['zero_cross_count'])
                harmonic.append(features['harmonic'])
                percussive.append(features['percussive'])
                amplitude.append(features['amplitude'])
                spectral_rolloff.append(features['spectral_rolloff'])
                mfcc.append(features['mfcc'])
                chroma_stft.append(features['chroma_stft'])
                rmse.append(features['rmse'])
                spec_bw.append(features['spec_bw'])
            img_df['energy'] = energy
            img_df['tempo'] = tempo
            img_df['zero_cross_count'] = zero_cross_count
            img_df['harmonic'] = harmonic
            img_df['percussive'] = percussive
            img_df['amplitude'] = amplitude
            img_df['spectral_rolloff'] = spectral_rolloff
            img_df['mfcc'] = mfcc
            img_df['chroma_stft'] = chroma_stft
            img_df['rmse'] = rmse
            img_df['spec_bw'] = spec_bw

            os.remove('newSong.wav')

            img_df = process_data(img_df)
            img_df.to_csv(img_info)
            print("done with", img_info) 

1196
403a42b604f504069c41d39c5a4fc2 0
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/403a42b604f504069c41d39c5a4fc2.csv
e45a4697e1e6be16f301f13197ffa4 1
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/e45a4697e1e6be16f301f13197ffa4.csv
11fe2dd4cccaf0567c878e19e1811a 2
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/11fe2dd4cccaf0567c878e19e1811a.csv
a5b2dd3ed41aabb5733c7cfffff0bc 3
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/a5b2dd3ed41aabb5733c7cfffff0bc.csv
80870ec50515107ba7034ab758cebe 4
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/80870ec50515107ba7034ab758cebe.csv
d66440e7524c0a8b9b206fbb7295ba 5
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/d66440e7524c0a8b9b206fbb7295ba.csv
231b80c884b2f1390e17f09a187889 6
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/231b80c884b2f1390e17f09a187889.csv
d50faca942c93b02863dc4cc86a839 7
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/d50faca942c93b02863dc4cc86a839.csv
4298fbb0147

pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/f8aae8248de6014a9116c59c024a98.csv
014f99c490a2e0bf55ab2e66a58233 67
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/014f99c490a2e0bf55ab2e66a58233.csv
1e9fe838b960ab31cdbb79855a9427 68
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/1e9fe838b960ab31cdbb79855a9427.csv
e9830ed5c29034c4120b88a8b48772 69
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/e9830ed5c29034c4120b88a8b48772.csv
ba54f9d576e93f2b0cf4622a92310f 70
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/ba54f9d576e93f2b0cf4622a92310f.csv
9914764b34cff2aac25a82a33d5494 71
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/9914764b34cff2aac25a82a33d5494.csv
bcd48e405eb0d32920e0e75675d46e 72
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/bcd48e405eb0d32920e0e75675d46e.csv
a1b2ac6681b22c01ff4586f32271e5 73
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/a1b2ac6681b22c01ff4586f32271e5.csv
4dc69aef6f70a7ec8722d3da439483 74
pass fil

pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/274869b912b256df69b9e7c5c0cb11.csv
31f08ef4a727a8943644261454f1fa 133
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/31f08ef4a727a8943644261454f1fa.csv
4516462b9495672b7477353d66aa97 134
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/4516462b9495672b7477353d66aa97.csv
00d3113bae64107671f55b51e183b9 135
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/00d3113bae64107671f55b51e183b9.csv
8b6e73994460c3b117787b63623a11 136
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/8b6e73994460c3b117787b63623a11.csv
e5bca888fec7f8c0116db0002a7077 137
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/e5bca888fec7f8c0116db0002a7077.csv
6b2603f375d426a5b1c28ab11a8af6 138
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/6b2603f375d426a5b1c28ab11a8af6.csv
895e6371fceaee140333d233fddfc5 139
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/895e6371fceaee140333d233fddfc5.csv
a82ccac57d0ca442727b47aaa95d01 140


pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/ccffca565b60e96ba17c4e8dc41781.csv
bc79162b1f6d9f4cae7d4746e04f5c 199
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/bc79162b1f6d9f4cae7d4746e04f5c.csv
09bf18e7baee13e303f4b77aeaadbc 200
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/09bf18e7baee13e303f4b77aeaadbc.csv
ff1d2aa6d4c4e552599a5f663a2eb3 201
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/ff1d2aa6d4c4e552599a5f663a2eb3.csv
35715e318743d24964298a6b2db87f 202
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/35715e318743d24964298a6b2db87f.csv
c6ffe9e1e01e0ea51c67abda8f81a9 203
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/c6ffe9e1e01e0ea51c67abda8f81a9.csv
94dcc6eed3fe7eb34c0c019e2550f2 204
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/94dcc6eed3fe7eb34c0c019e2550f2.csv
fb7cf9bf7c4eb77941d69ae7303c4a 205
pass file:  /Volumes/Seagate/natasha-diploma/videoinfo/fb7cf9bf7c4eb77941d69ae7303c4a.csv
941822c7be0028345cb001fb7b6f27 206


/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

done with /Volumes/Seagate/natasha-diploma/videoinfo/acbb334f31c86f9e9b274650e7cf8e.csv
749dd5f8902dbc94e9a1d5856c497c 853
done with /Volumes/Seagate/natasha-diploma/videoinfo/749dd5f8902dbc94e9a1d5856c497c.csv
01bc55dad49dc1dfa856c85edbf6d1 854
done with /Volumes/Seagate/natasha-diploma/videoinfo/01bc55dad49dc1dfa856c85edbf6d1.csv
268c8018b45e2713efb5881f81751a 855
done with /Volumes/Seagate/natasha-diploma/videoinfo/268c8018b45e2713efb5881f81751a.csv
783a7e3724295d1ed4565fef304ddf 856
done with /Volumes/Seagate/natasha-diploma/videoinfo/783a7e3724295d1ed4565fef304ddf.csv
a389a5c36a5a66e3643f202bdaa318 857
done with /Volumes/Seagate/natasha-diploma/videoinfo/a389a5c36a5a66e3643f202bdaa318.csv
ef2374bb5caec2712d6796604247ae 858
done with /Volumes/Seagate/natasha-diploma/videoinfo/ef2374bb5caec2712d6796604247ae.csv
a6caf80f7e1afa96600ae6887a56a2 859
done with /Volumes/Seagate/natasha-diploma/videoinfo/a6caf80f7e1afa96600ae6887a56a2.csv
85872df83a4b8f0f0b72178b1e27f6 860
done with /Volum

done with /Volumes/Seagate/natasha-diploma/videoinfo/01c8ee28cc66e7d7801786ba6c8d5f.csv
a81ba0d2920536f534033daa15942c 920
done with /Volumes/Seagate/natasha-diploma/videoinfo/a81ba0d2920536f534033daa15942c.csv
509b481f553360ce418fbc4bb4f5e0 921
done with /Volumes/Seagate/natasha-diploma/videoinfo/509b481f553360ce418fbc4bb4f5e0.csv
0d2848d46002183dbd65981e735f49 922
done with /Volumes/Seagate/natasha-diploma/videoinfo/0d2848d46002183dbd65981e735f49.csv
b27b224dba38acf9a99701f250eb8a 923
done with /Volumes/Seagate/natasha-diploma/videoinfo/b27b224dba38acf9a99701f250eb8a.csv
1ff70bf7a15193b4363be70dd38702 924
done with /Volumes/Seagate/natasha-diploma/videoinfo/1ff70bf7a15193b4363be70dd38702.csv
153068b35776cd04068215f6d45aa4 925
done with /Volumes/Seagate/natasha-diploma/videoinfo/153068b35776cd04068215f6d45aa4.csv
2de54d71c932a282478b9fc2e2100f 926
done with /Volumes/Seagate/natasha-diploma/videoinfo/2de54d71c932a282478b9fc2e2100f.csv
d1a994082b70e143ebe89de97700a8 927
done with /Volum

done with /Volumes/Seagate/natasha-diploma/videoinfo/cedcd49fc6c7319a868a71a9267f31.csv
660510cf9d088c4dbcb713e93b774d 987
done with /Volumes/Seagate/natasha-diploma/videoinfo/660510cf9d088c4dbcb713e93b774d.csv
b1165d9413566f39f1df121d9d7bee 988
done with /Volumes/Seagate/natasha-diploma/videoinfo/b1165d9413566f39f1df121d9d7bee.csv
1793ea61c312bd89531bde97cf415a 989
done with /Volumes/Seagate/natasha-diploma/videoinfo/1793ea61c312bd89531bde97cf415a.csv
38a1b2d1bb6603ab05014d67b59f5d 990
done with /Volumes/Seagate/natasha-diploma/videoinfo/38a1b2d1bb6603ab05014d67b59f5d.csv
52845cad4c1fe68e325c22957c276d 991
done with /Volumes/Seagate/natasha-diploma/videoinfo/52845cad4c1fe68e325c22957c276d.csv
968528d82fbfbaba35d28256c0f607 992
done with /Volumes/Seagate/natasha-diploma/videoinfo/968528d82fbfbaba35d28256c0f607.csv
7f2b65600b2d086e26bfaf44fc0821 993
done with /Volumes/Seagate/natasha-diploma/videoinfo/7f2b65600b2d086e26bfaf44fc0821.csv
eab92ba38bec1834e6839ba5366bfc 994
done with /Volum

f1f8c98e9888b7bd329b381a1cd2c0 1053
done with /Volumes/Seagate/natasha-diploma/videoinfo/f1f8c98e9888b7bd329b381a1cd2c0.csv
309d740c1873d9806ad81cff8a7a6c 1054
done with /Volumes/Seagate/natasha-diploma/videoinfo/309d740c1873d9806ad81cff8a7a6c.csv
0144ab4375ae796e3c8e5fc162ed0e 1055
done with /Volumes/Seagate/natasha-diploma/videoinfo/0144ab4375ae796e3c8e5fc162ed0e.csv
0592d6a5e00fd8712e8f401b5418a7 1056
done with /Volumes/Seagate/natasha-diploma/videoinfo/0592d6a5e00fd8712e8f401b5418a7.csv
cfaf54aaa56d770630e8bd921eec92 1057
done with /Volumes/Seagate/natasha-diploma/videoinfo/cfaf54aaa56d770630e8bd921eec92.csv
36b656ae67ffb9d0ad009ea06e5a24 1058
done with /Volumes/Seagate/natasha-diploma/videoinfo/36b656ae67ffb9d0ad009ea06e5a24.csv
18a88e2f4348b624c03acffe3808b8 1059
done with /Volumes/Seagate/natasha-diploma/videoinfo/18a88e2f4348b624c03acffe3808b8.csv
c00c96e331ae1a0b741b332a6eea57 1060
done with /Volumes/Seagate/natasha-diploma/videoinfo/c00c96e331ae1a0b741b332a6eea57.csv
ae6e739d

done with /Volumes/Seagate/natasha-diploma/videoinfo/1db7aaaf01d6e39e252d4361d916a8.csv
57a6c27ac59feef428160b724002b5 1120
done with /Volumes/Seagate/natasha-diploma/videoinfo/57a6c27ac59feef428160b724002b5.csv
3574ce6f8b51fa787f8054a2734c59 1121
done with /Volumes/Seagate/natasha-diploma/videoinfo/3574ce6f8b51fa787f8054a2734c59.csv
5d9b657b32bff89dedf7c9480ba3e6 1122
done with /Volumes/Seagate/natasha-diploma/videoinfo/5d9b657b32bff89dedf7c9480ba3e6.csv
a34ad1ab6cdb98b8606932a0cfc231 1123
done with /Volumes/Seagate/natasha-diploma/videoinfo/a34ad1ab6cdb98b8606932a0cfc231.csv
127c45352d50fad1cd1936b2e21354 1124
done with /Volumes/Seagate/natasha-diploma/videoinfo/127c45352d50fad1cd1936b2e21354.csv
d33827f0190f1df36825d916828df7 1125
done with /Volumes/Seagate/natasha-diploma/videoinfo/d33827f0190f1df36825d916828df7.csv
fbaaad58d8b0bc011a639d40ee8778 1126
done with /Volumes/Seagate/natasha-diploma/videoinfo/fbaaad58d8b0bc011a639d40ee8778.csv
a1a39ec242c9d607fd9e9132c9e785 1127
done wit